# measure the average minimum distance between two lines, or the zonal stats around each line's buffer

In [3]:
from geopandas import read_file, GeoSeries, GeoDataFrame
from shapely.geometry import Point
from functools import reduce
import numpy as np
from rioxarray import open_rasterio
from rasterstats import zonal_stats
from operator import mul
from pathlib import Path
from xarray import DataArray
from pandas import DataFrame
from shapely.geometry import LineString, Polygon
from shapely.validation import make_valid

from src.rules2weights import write_compressed

# 5 m resolution
## distances

In [6]:
def compute_line_distances(line1: LineString, line2: LineString) -> list:
    return [Point(p).distance(line2) for p in line1.coords]

In [48]:
least_cost_path_5_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_false.json")
least_cost_path_5_false = least_cost_path_5_false.to_crs(25832)
least_cost_path_5_true = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_true.gpkg")

In [49]:
least_cost_path_5_false.geometry.values[0].length, least_cost_path_5_true.geometry.values[0].length

(76136.34083614746, 78002.0066579491)

In [50]:
distances_5  = compute_line_distances(least_cost_path_5_false.geometry.values[0], least_cost_path_5_true.geometry.values[0])

In [51]:
mean_min_distances_5 =  np.mean(distances_5)
mean_min_distances_5

126.04147538653905

In [52]:
max_min_distances_5 =  np.max(distances_5)
max_min_distances_5

1065.0048701083288

## costs

In [19]:
least_cost_path_5_false['total cost'][0], least_cost_path_5_true['total cost'][0]

(18665.923, 19516.754)

In [20]:
least_cost_path_5_false['total cost'][0]*5, least_cost_path_5_true['total cost'][0]*5

(93329.61499999999, 97583.77)

In [21]:
least_cost_path_5_false['total cost'][0]*5 - least_cost_path_5_true['total cost'][0]*5

-4254.155000000013

In [22]:
least_cost_path_5_false.geometry.length[0], least_cost_path_5_true.geometry.length[0]

(59873.38154014804, 78002.0066579491)

In [23]:
(least_cost_path_5_false['total cost'][0]*5 - least_cost_path_5_true['total cost'][0]*5) / ((least_cost_path_5_false.geometry.length + least_cost_path_5_true.geometry.length)/2)

0   -0.06171
dtype: float64

In [24]:
least_cost_path_5_false['total cost'][0] - least_cost_path_5_true['total cost'][0]

-850.831000000002

## zonal stats

In [40]:
def compute_zonal_stat_path(base_raster: DataArray, path: GeoSeries, buffer: float, cmap):
    stats = zonal_stats(path.buffer(buffer), base_raster.data[0],
                        affine= base_raster.rio.transform(), nodata=base_raster.rio.nodata,
                        categorical=True, category_map=cmap)[0]

    stat_sum = np.sum(list(stats.values()))
    rel_stats = {k: (v / stat_sum * 100.0) for k, v in stats.items()}
    return rel_stats

In [41]:
buffer_dist_100 = 100
raster_5f = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_False.tif'))
cmap = {0.1: 'Preferential', 0.5: 'No Restriction', 5.0: 'Restricted', 10.0: 'Strongly Restricted', 500: 'Prohibited'}

rel_stats_5f_100 = compute_zonal_stat_path(raster_5f, least_cost_path_5_false, buffer_dist_100, cmap)
rel_stats_5f_100

{'Preferential': 4.700294041698944,
 'No Restriction': 58.703592639794756,
 'Restricted': 8.782100940433649,
 'Strongly Restricted': 0.7344795128736954,
 'Prohibited': 27.079532865198956}

In [44]:
buffer_dist_5 = 5

rel_stats_5f_5 = compute_zonal_stat_path(raster_5f, least_cost_path_5_false, buffer_dist_5, cmap)
rel_stats_5f_5

{'Preferential': 5.394346791365705,
 'No Restriction': 58.85766773829902,
 'Restricted': 8.35873241200785,
 'Strongly Restricted': 0.6847313264581855,
 'Prohibited': 26.704521731869235}

In [46]:
raster_5t = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_True.tif'))

rel_stats_5t_100 = compute_zonal_stat_path(raster_5t, least_cost_path_5_true, buffer_dist_100, cmap)
rel_stats_5t_100

{'Preferential': 18.89687706997108,
 'No Restriction': 67.30923954311194,
 'Restricted': 1.291177278381828,
 'Strongly Restricted': 0.9866390939787065,
 'Prohibited': 11.51606701455644}

In [48]:
raster_5t = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_True.tif'))

rel_stats_5t_5 = compute_zonal_stat_path(raster_5t, least_cost_path_5_true, buffer_dist_5, cmap)
rel_stats_5t_5

{'Preferential': 28.469760979897362,
 'No Restriction': 66.44078369106013,
 'Restricted': 1.6189855113555813,
 'Strongly Restricted': 0.4961864526921659,
 'Prohibited': 2.9742833649947547}

## lines -> buffered Polygon

In [77]:
def create_buffer_polygon(line1: LineString, line2: LineString, crs, buffer) -> GeoDataFrame:
    x = list(line1.xy[0])
    y = list(line1.xy[1])

    x.extend(list(reversed(line2.xy[0])))
    y.extend(list(reversed(line2.xy[1])))

    poly = make_valid(Polygon(zip(x, y))).buffer(buffer)

    return GeoDataFrame({'buffer': [buffer, ], 'geometry': [poly, ]}, crs=crs)

In [84]:
create_buffer_polygon(least_cost_path_5_false.geometry.values[0], least_cost_path_5_true.geometry.values[0], 25832, 2*max_min_distances_5).to_file("./../results/Buffer_Polygons/least_cost_path_res_5_buffer.gpkg")

# 10 m resolution
## distances

In [10]:
least_cost_path_10_false = read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_false.gpkg")
least_cost_path_10_true = read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_true.gpkg")
least_cost_path_10_false.geometry.values[0].length, least_cost_path_10_true.geometry.values[0].length

(75430.10086958889, 77936.56874848259)

In [11]:
distances_10  =  compute_line_distances(least_cost_path_10_false.geometry.values[0], least_cost_path_10_true.geometry.values[0])
mean_min_distances_10 =  np.mean(distances_10)
mean_min_distances_10

277.921414632231

In [12]:
max_min_distances_10 =  np.max(distances_10)
max_min_distances_10

1590.0

## costs

In [52]:
least_cost_path_10_false['total cost'][0], least_cost_path_10_true['total cost'][0]

(8931.245, 9731.175)

In [53]:
least_cost_path_10_false['total cost'][0]*10, least_cost_path_10_true['total cost'][0]*10

(89312.45000000001, 97311.75)

In [54]:
least_cost_path_10_false['total cost'][0]*10 - least_cost_path_10_true['total cost'][0]*10

-7999.299999999988

In [55]:
least_cost_path_10_false.geometry.length

0    75430.10087
dtype: float64

In [56]:
(least_cost_path_10_false['total cost'][0]*10 - least_cost_path_10_true['total cost'][0]*10) / ((least_cost_path_10_false.geometry.length + least_cost_path_10_true.geometry.length)/2)

0   -0.104316
dtype: float64

In [57]:
least_cost_path_10_false['total cost'][0] - least_cost_path_10_true['total cost'][0]

-799.9299999999985

## Zonal Stats

In [58]:
raster_10_f = open_rasterio(Path(r'./../results/weights/result_res_10_all_touched_False.tif')).rio.reproject_match(raster_5f)

rel_stats_10f_100 = compute_zonal_stat_path(raster_10_f, least_cost_path_10_false, buffer_dist_100, cmap)
rel_stats_10f_100

{'Preferential': 19.595563054409382,
 'No Restriction': 68.53326873924593,
 'Restricted': 0.9772639949215683,
 'Strongly Restricted': 0.7993518317435392,
 'Prohibited': 10.094552379679591}

In [60]:
rel_stats_10f_5 = compute_zonal_stat_path(raster_10_f, least_cost_path_10_false, buffer_dist_5, cmap)
rel_stats_10f_5

{'Preferential': 33.54139715394567,
 'No Restriction': 64.47930142302717,
 'Restricted': 0.7309184993531695,
 'Strongly Restricted': 0.3169469598965071,
 'Prohibited': 0.9314359637774904}

In [62]:
raster_10_t = open_rasterio(Path(r'./../results/weights/result_res_10_all_touched_True.tif')).rio.reproject_match(raster_5t)

rel_stats_10t_100 = compute_zonal_stat_path(raster_10_t, least_cost_path_10_true, buffer_dist_100, cmap)
rel_stats_10t_100

{'Preferential': 18.921619271395883,
 'No Restriction': 66.60880238754925,
 'Restricted': 1.6271089829435321,
 'Strongly Restricted': 1.3743306069110028,
 'Prohibited': 11.468138751200332}

In [63]:
rel_stats_10t_5 = compute_zonal_stat_path(raster_10_t, least_cost_path_10_true, buffer_dist_5, cmap)
rel_stats_10t_5

{'Preferential': 33.6669071744518,
 'No Restriction': 63.399943532954794,
 'Restricted': 1.3771684913887756,
 'Strongly Restricted': 0.5803557423847915,
 'Prohibited': 0.9756250588198387}

## lines -> buffered Polygons

In [85]:
create_buffer_polygon(least_cost_path_10_false.geometry.values[0], least_cost_path_10_true.geometry.values[0], 25832, 2*max_min_distances_10).to_file("./../results/Buffer_Polygons/least_cost_path_res_10_buffer.gpkg")

# 25 m resolution
## Distances

In [14]:
least_cost_path_25_false = read_file("../results/least_cost_paths/least_cost_path_test_points_res_25_al_false.gpkg")
least_cost_path_25_true = read_file("../results/least_cost_paths/least_cost_path_test_points_res_25_al_true.gpkg")
least_cost_path_25_false.geometry.values[0].length, least_cost_path_25_true.geometry.values[0].length

(75422.85413306313, 78422.8541330632)

In [15]:
distances_25  =  compute_line_distances(least_cost_path_25_false.geometry.values[0], least_cost_path_25_true.geometry.values[0])
mean_min_distances_25 =  np.mean(distances_25)
mean_min_distances_25

313.75138750272674

In [16]:
max_min_distances_25 =  np.max(distances_25)
max_min_distances_25

1621.1492836873476

## Costs

In [66]:
least_cost_path_25_false['total cost'][0], least_cost_path_25_true['total cost'][0]

(3354.869, 3872.656)

In [67]:
least_cost_path_25_false['total cost'][0]*25, least_cost_path_25_true['total cost'][0]*25

(83871.725, 96816.4)

In [68]:
least_cost_path_25_false['total cost'][0]*25 - least_cost_path_25_true['total cost'][0]*25

-12944.674999999988

In [69]:
least_cost_path_25_false.geometry.length

0    75422.854133
dtype: float64

In [70]:
(least_cost_path_25_false['total cost'][0]*25 - least_cost_path_25_true['total cost'][0]*25) / ((least_cost_path_25_false.geometry.length + least_cost_path_25_true.geometry.length)/2)

0   -0.168281
dtype: float64

In [71]:
least_cost_path_25_false['total cost'][0] - least_cost_path_25_true['total cost'][0]

-517.7869999999998

## Zonal stat

In [72]:
raster_25f = open_rasterio(Path(r'./../results/weights/result_res_25_all_touched_False.tif')).rio.reproject_match(raster_5f)

rel_stats_25f_100 = compute_zonal_stat_path(raster_25f, least_cost_path_25_false, buffer_dist_100, cmap)
rel_stats_25f_100

{'Preferential': 19.826571848213025,
 'No Restriction': 68.87454257287716,
 'Restricted': 0.9564272141132344,
 'Strongly Restricted': 0.6668270946220677,
 'Prohibited': 9.675631270174508}

In [74]:
rel_stats_25f_5 = compute_zonal_stat_path(raster_25f, least_cost_path_25_false, buffer_dist_5, cmap)
rel_stats_25f_5

{'Preferential': 34.22749624569359,
 'No Restriction': 64.91269397249788,
 'Restricted': 0.21495244545213626,
 'Strongly Restricted': 0.09128117546597568,
 'Prohibited': 0.5535761608904332}

In [76]:
raster_25t = open_rasterio(Path(r'./../results/weights/result_res_25_all_touched_True.tif')).rio.reproject_match(raster_5t)

rel_stats_25t_100 = compute_zonal_stat_path(raster_25t, least_cost_path_25_true, buffer_dist_100, cmap)
rel_stats_25t_100

{'Preferential': 18.67715636303323,
 'No Restriction': 65.45959670062581,
 'Restricted': 1.9550961041612178,
 'Strongly Restricted': 2.541034850118336,
 'Prohibited': 11.367115982061406}

In [78]:
rel_stats_25t_5 = compute_zonal_stat_path(raster_25t, least_cost_path_25_true, buffer_dist_5, cmap)
rel_stats_25t_5

{'Preferential': 31.925548785577163,
 'No Restriction': 65.51484294577526,
 'Restricted': 1.2825863172244945,
 'Strongly Restricted': 0.7038922738780847,
 'Prohibited': 0.5731296775450018}

## lines -> Polygones

In [86]:
create_buffer_polygon(least_cost_path_25_false.geometry.values[0], least_cost_path_25_true.geometry.values[0], 25832, 2*max_min_distances_25).to_file("./../results/Buffer_Polygons/least_cost_path_res_25_buffer.gpkg")

# 50 m resolution
## Distances

In [17]:
least_cost_path_50_false = read_file("../results/least_cost_paths/least_cost_path_test_points_res_50_al_false.gpkg")
least_cost_path_50_true = read_file("../results/least_cost_paths/least_cost_path_test_points_res_50_al_true.gpkg")
least_cost_path_50_false.geometry.values[0].length, least_cost_path_50_true.geometry.values[0].length

(76135.0213422174, 70619.94558270913)

In [18]:
distances_50  =  compute_line_distances(least_cost_path_50_false.geometry.values[0], least_cost_path_50_true.geometry.values[0])
mean_min_distances_50 =  np.mean(distances_50)
mean_min_distances_50

1140.0069288754526

In [19]:
max_min_distances_50 =  np.max(distances_50)
max_min_distances_50

4950.0

## costs

In [81]:
least_cost_path_50_false['total cost'][0], least_cost_path_50_true['total cost'][0]

(1409.023, 2300.073)

In [82]:
least_cost_path_50_false['total cost'][0]*50, least_cost_path_50_true['total cost'][0]*50

(70451.15, 115003.65)

In [83]:
least_cost_path_50_false['total cost'][0]*50 - least_cost_path_50_true['total cost'][0]*50

-44552.5

In [84]:
least_cost_path_50_false.geometry.length

0    76135.021342
dtype: float64

In [85]:
(least_cost_path_50_false['total cost'][0]*50 - least_cost_path_50_true['total cost'][0]*50) / ((least_cost_path_50_false.geometry.length + least_cost_path_50_true.geometry.length)/2)

0   -0.607169
dtype: float64

In [86]:
least_cost_path_50_false['total cost'][0] - least_cost_path_50_true['total cost'][0]

-891.05

## Zonal stat

In [87]:
raster_50f = open_rasterio(Path(r'./../results/weights/result_res_50_all_touched_False.tif')).rio.reproject_match(raster_5f)

rel_stats_50f_100 = compute_zonal_stat_path(raster_50f, least_cost_path_50_false, buffer_dist_100, cmap)
rel_stats_50f_100

{'Preferential': 20.424131915614257,
 'No Restriction': 67.97554122198528,
 'Restricted': 0.8699340849414755,
 'Strongly Restricted': 0.6704954069913697,
 'Prohibited': 10.05989737046762}

In [88]:
rel_stats_50f_5 = compute_zonal_stat_path(raster_50f, least_cost_path_50_false, buffer_dist_5, cmap)
rel_stats_50f_5

{'Preferential': 33.180814641794385,
 'No Restriction': 66.19844494928488,
 'Restricted': 0.07679262790772086,
 'Strongly Restricted': 0.0415960067833488,
 'Prohibited': 0.5023517742296739}

In [89]:
raster_50t = open_rasterio(Path(r'./../results/weights/result_res_50_all_touched_True.tif')).rio.reproject_match(raster_5t)

rel_stats_50t_100 = compute_zonal_stat_path(raster_50t, least_cost_path_50_true, buffer_dist_100, cmap)
rel_stats_50t_100

{'Preferential': 9.144355315406148,
 'No Restriction': 75.68238741961079,
 'Restricted': 3.8982662735442966,
 'Strongly Restricted': 4.172339545542493,
 'Prohibited': 7.102651445896267}

In [90]:
rel_stats_50t_5 = compute_zonal_stat_path(raster_50t, least_cost_path_50_true, buffer_dist_5, cmap)
rel_stats_50t_5

{'Preferential': 13.028229549908882,
 'No Restriction': 82.98662448853283,
 'Restricted': 1.9564694151222366,
 'Strongly Restricted': 1.5919953237286386,
 'Prohibited': 0.43668122270742354}

## lines -> Polygon


In [87]:
create_buffer_polygon(least_cost_path_50_false.geometry.values[0], least_cost_path_50_true.geometry.values[0], 25832, 2*max_min_distances_50).to_file("./../results/Buffer_Polygons/least_cost_path_res_50_buffer.gpkg")

# 100 m resolution
## Distances

In [21]:
least_cost_path_100_false = read_file("../results/least_cost_paths/least_cost_path_test_points_res_100_al_false.gpkg")
least_cost_path_100_true = read_file("../results/least_cost_paths/least_cost_path_test_points_res_100_al_true.gpkg")
least_cost_path_100_false.geometry.values[0].length, least_cost_path_100_true.geometry.values[0].length

(76283.80711525407, 74120.73210305478)

In [22]:
distances_100  =  compute_line_distances(least_cost_path_100_false.geometry.values[0], least_cost_path_100_true.geometry.values[0])
mean_min_distances_100 =  np.mean(distances_100)
mean_min_distances_100

1946.410723103447

In [23]:
max_min_distances_100 =  np.max(distances_100)
max_min_distances_100

6016.643582596529

## costs

In [93]:
least_cost_path_100_false['total cost'][0], least_cost_path_100_true['total cost'][0]

(640.516, 1572.268)

In [94]:
least_cost_path_100_false['total cost'][0]*100, least_cost_path_100_true['total cost'][0]*100

(64051.6, 157226.8)

In [95]:
least_cost_path_100_false['total cost'][0]*100 - least_cost_path_100_true['total cost'][0]*100

-93175.19999999998

In [96]:
(least_cost_path_100_false['total cost'][0]*100 - least_cost_path_100_true['total cost'][0]*199) / ((least_cost_path_100_false.geometry.length + least_cost_path_100_true.geometry.length)/2)

0   -3.308806
dtype: float64

In [97]:
least_cost_path_100_false['total cost'][0] - least_cost_path_100_true['total cost'][0]

-931.7520000000001

## Zonal Stat

In [98]:
raster_100f = open_rasterio('./../results/weights/result_res_100_all_touched_False.tif').rio.reproject_match(raster_50f)

rel_stats_100f_100 = compute_zonal_stat_path(raster_100f, least_cost_path_100_false, buffer_dist_100, cmap)
rel_stats_100f_100

{'Preferential': 21.116438041932376,
 'No Restriction': 69.1258298979545,
 'Restricted': 1.105329042659868,
 'Strongly Restricted': 0.7209380058854757,
 'Prohibited': 7.931465011567777}

In [99]:
rel_stats_100f_100 = compute_zonal_stat_path(raster_100f, least_cost_path_100_false, buffer_dist_5, cmap)
rel_stats_100f_100

{'Preferential': 30.733593200843508,
 'No Restriction': 68.81270368713656,
 'Restricted': 0.03834110805802288,
 'Strongly Restricted': 0.01917055402901144,
 'Prohibited': 0.39619144993290306}

In [100]:
raster_100t = open_rasterio(r'./../results/weights/result_res_100_all_touched_True.tif').rio.reproject_match(raster_5t)

rel_stats_100t_100 = compute_zonal_stat_path(raster_100t, least_cost_path_100_true, buffer_dist_100, cmap)
rel_stats_100t_100

{'Preferential': 6.9728627271334,
 'No Restriction': 73.80772333556207,
 'Restricted': 5.540126549660721,
 'Strongly Restricted': 8.51254465195475,
 'Prohibited': 5.166742735689059}

In [101]:
rel_stats_100t_5 = compute_zonal_stat_path(raster_100t, least_cost_path_100_true, buffer_dist_5, cmap)
rel_stats_100t_5

{'Preferential': 10.132879492046868,
 'No Restriction': 81.94344439353276,
 'Restricted': 3.9111082018720955,
 'Strongly Restricted': 3.610001963736335,
 'Prohibited': 0.40256594881193947}

## lines -> buffered Polygons

In [88]:
create_buffer_polygon(least_cost_path_100_false.geometry.values[0], least_cost_path_100_true.geometry.values[0], 25832, 2 * max_min_distances_100).to_file("./../results/Buffer_Polygons/least_cost_path_res_100_buffer.gpkg")

# Mask the 2.5 m and 5 m resolution Raster by the Buffer (from the 50 m resolution Paths)
Masking the original 50 m resoltion cost raster by the polygon, created from the buffered 50 m resolution least cost path.
The buffer is two times the maximum minimum distance between the paths.
## masking the 5 m resolution raster

In [7]:
mask_50 = read_file("./../results/Buffer_Polygons/least_cost_path_res_50_buffer.gpkg")

raster_5f = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_False.tif'))
raster_5t = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_True.tif'))

raster_5f_clipped = raster_5f.rio.clip(mask_50.geometry.values, mask_50.crs, drop=False)
write_compressed(raster_5f_clipped, Path(r'./../results/weights/Clipped/result_res_5_all_touched_False_clipped.tif'))
raster_5f_clipped_dropped = raster_5f.rio.clip(mask_50.geometry.values, mask_50.crs, drop=True)
write_compressed(raster_5f_clipped_dropped, Path(r'./../results/weights/Clipped/result_res_5_all_touched_False_clipped_drop.tif'))

raster_5t_clipped = raster_5t.rio.clip(mask_50.geometry.values, mask_50.crs, drop=False)
write_compressed(raster_5t_clipped, Path(r'./../results/weights/Clipped/result_res_5_all_touched_True_clipped.tif'))
raster_5t_clipped_dropped = raster_5t.rio.clip(mask_50.geometry.values, mask_50.crs, drop=True)
write_compressed(raster_5t_clipped_dropped, Path(r'./../results/weights/Clipped/result_res_5_all_touched_True_clipped_drop.tif'))

rasters = [raster_5f, raster_5f_clipped, raster_5f_clipped_dropped, raster_5t, raster_5t_clipped, raster_5t_clipped_dropped]

raster_y = np.array([x.shape[1] for x in rasters])
raster_x = np.array([x.shape[2] for x in rasters])
counts = raster_y * raster_x
nodata_counts = np.array([np.sum(x.data == x.rio.nodata) for x in rasters])


DataFrame(data={'height': raster_y,
                'width': raster_x,
                'pixel_count': counts,
                'pixel_percentage': counts / np.max(counts) * 100.0,
                'nodata_count': nodata_counts,
                'nodata_percentage': nodata_counts / counts * 100.0,
                'kept_percentage': (counts - nodata_counts) / np.max(counts) * 100.0},

          index=['5f_original', '5f_clip', '5f_clip_drop', '5t_original', '5t_clip', '5t_clip_drop'])

,height,width,pixel_count,pixel_percentage,nodata_count,nodata_percentage,kept_percentage
5f_original,17481,11270,197010870,100.000000,75314390,38.228545,61.771455
5f_clip,17481,11270,197010870,100.000000,150122228,76.199972,23.800028
5f_clip_drop,12832,9465,121454880,61.648822,74566238,61.394189,23.800028
5t_original,17481,11270,197010870,100.000000,75274817,38.208459,61.791541
5t_clip,17481,11270,197010870,100.000000,150110945,76.194245,23.805755
5t_clip_drop,12832,9465,121454880,61.648822,74554955,61.384899,23.805755


2.5 m

In [ ]:
mask_50 = read_file("./../results/Buffer_Polygons/least_cost_path_res_50_buffer.gpkg")

raster_2p5f = open_rasterio(Path(r'./../results/weights/result_res_2.5_all_touched_False.tif'))
raster_2p5t = open_rasterio(Path(r'./../results/weights/result_res_2.5_all_touched_True.tif'))

raster_2p5f_clipped = raster_5f.rio.clip(mask_50.geometry.values, mask_50.crs, drop=False)
write_compressed(raster_2p5f_clipped, Path(r'./../results/weights/Clipped/result_res_2p5_all_touched_False_clipped.tif'))
raster_2p5f_clipped_dropped = raster_5f.rio.clip(mask_50.geometry.values, mask_50.crs, drop=True)
write_compressed(raster_2p5f_clipped_dropped, Path(r'./../results/weights/Clipped/result_res_2p5_all_touched_False_clipped_drop.tif'))

raster_2p5t_clipped = raster_5t.rio.clip(mask_50.geometry.values, mask_50.crs, drop=False)
write_compressed(raster_2p5t_clipped, Path(r'./../results/weights/Clipped/result_res_2p5_all_touched_True_clipped.tif'))
raster_2p5t_clipped_dropped = raster_5t.rio.clip(mask_50.geometry.values, mask_50.crs, drop=True)
write_compressed(raster_2p5t_clipped_dropped, Path(r'./../results/weights/Clipped/result_res_2p5_all_touched_True_clipped_drop.tif'))

rasters = [raster_2p5f, raster_2p5f_clipped, raster_2p5f_clipped_dropped, raster_2p5t, raster_2p5t_clipped, raster_2p5t_clipped_dropped]

raster_y = np.array([x.shape[1] for x in rasters])
raster_x = np.array([x.shape[2] for x in rasters])
counts = raster_y * raster_x
nodata_counts = np.array([np.sum(x.data == x.rio.nodata) for x in rasters])


DataFrame(data={'height': raster_y,
                'width': raster_x,
                'pixel_count': counts,
                'pixel_percentage': counts / np.max(counts) * 100.0,
                'nodata_count': nodata_counts,
                'nodata_percentage': nodata_counts / counts * 100.0,
                'kept_percentage': (counts - nodata_counts) / np.max(counts) * 100.0},

          index=['5f_original', '5f_clip', '5f_clip_drop', '5t_original', '5t_clip', '5t_clip_drop'])

# Mask the 2.5 and 5 m resolution raster by the non-reachable pixels

In [1]:
# TODO

# Distances all touched False

In [102]:
distances_false  =  np.array([Point(p).distance(least_cost_path_100_false.geometry.values[0]) for p in least_cost_path_5_false.geometry.values[0].coords])
mean_min_distances_false =  np.mean(distances_false)
mean_min_distances_false

243.4202763963724

# Distances all touched True


In [103]:
distances_true  =  np.array([Point(p).distance(least_cost_path_100_true.geometry.values[0]) for p in least_cost_path_5_true.geometry.values[0].coords])
mean_min_distances_true =  np.mean(distances_true)
mean_min_distances_true

2109.436322189748